In [1]:
import numpy as np
import os
import math
# from src.utils import pkl_load, pkl_dump
import multiprocessing
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import time

In [2]:
df = pd.read_csv('Train_1_MHCps.txt')
df

,Sequence,BA,MHC_name,pseudoseq
0,DLDKKETVWHLEE,0.000000,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
1,HSLGKWLGHPDKF,0.000000,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
2,VPDHVVWSLFNTL,0.871874,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
3,HTGREIVDLMCHAT,0.000000,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
4,AAASVPAADKFKTFE,0.203668,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
...,...,...,...,...
106944,FEAQGAKANIAVD,0.000000,H-2-IEk,QEFFIASGAAVDAVMECSLVYFDFQKETVHIFFL
106945,EEDIEIIPIQEEEY,0.072792,H-2-IEk,QEFFIASGAAVDAVMECSLVYFDFQKETVHIFFL
106946,AAHSAAFEDLRVSSY,0.052982,H-2-IEk,QEFFIASGAAVDAVMECSLVYFDFQKETVHIFFL
106947,YAGIRRDGLLLRLVD,0.441317,H-2-IEk,QEFFIASGAAVDAVMECSLVYFDFQKETVHIFFL


In [3]:
def encode(sequence, max_len=None, encoding='onehot', pad_scale=None):
    """
    encodes a single peptide into a matrix, using 'onehot' or 'blosum'
    if 'blosum', then need to provide the blosum dictionary as argument
    """
    assert encoding in encoding_matrix_dict.keys(), f'Wrong encoding key {encoding} passed!'\
                                                    f'Should be any of {encoding_matrix_dict.keys()}'
    # One hot encode by setting 1 to positions where amino acid is present, 0 elsewhere
    size = len(sequence)
    blosum_matrix = encoding_matrix_dict[encoding]
    if encoding == 'onehot':
        int_encoded = [CHAR_TO_INT[char] for char in sequence]
        onehot_encoded = list()
        for value in int_encoded:
            letter = [0 for _ in range(len(AA_KEYS))]
            letter[value] = 1 if value != -1 else 0
            onehot_encoded.append(letter)
        tmp = np.array(onehot_encoded)

    # BLOSUM encode
    else:
        if blosum_matrix is None or not isinstance(blosum_matrix, dict):
            raise Exception('No BLOSUM matrix provided!')

        tmp = np.zeros([size, len(AA_KEYS)], dtype=np.float32)
        for idx in range(size):
            # Here, the way Morten takes cares of Xs is to leave it blank, i.e. as zeros
            # So only use blosum matrix to encode if sequence[idx] != 'X'
            if sequence[idx]!='X':
                tmp[idx, :] = blosum_matrix[sequence[idx]]


    # Padding if max_len is provided
    if max_len is not None and max_len > size:
        diff = int(max_len) - int(size)
        try:
            if pad_scale is None:
                pad_scale = 0 if encoding=='onehot' else -12
            tmp = np.concatenate([tmp, pad_scale*np.ones([diff, len(AA_KEYS)], dtype=np.float32)],
                                 axis=0)
        except:
            print('Here in encode', type(tmp), tmp.shape, len(AA_KEYS), type(diff), type(max_len), type(size), sequence)
            #     return tmp, diff, len(AA_KEYS)
            raise Exception
    return torch.from_numpy(tmp).float()


In [4]:
def encode_batch(sequences, max_len=None, encoding='onehot', pad_scale=None):
    """
    Encode multiple sequences at once.
    """
    if max_len is None:
        max_len = max([len(x) for x in sequences])
    # Contiguous to allow for .view operation
    return torch.stack([encode(seq, max_len, encoding, pad_scale) for seq in sequences]).contiguous()

In [5]:
DATADIR = '/Users/minij/OneDrive/Documentos/GitHub/PyNNalign/data/'
OUTDIR = '/Users/minij/OneDrive/Documentos/Try_output/'
# Stupid hardcoded variable
CNN_FEATS = ['EL_ratio', 'anchor_mutation', 'delta_VHSE1', 'delta_VHSE3', 'delta_VHSE7', 'delta_VHSE8',
             'delta_aliphatic_index',
             'delta_boman', 'delta_hydrophobicity', 'delta_isoelectric_point', 'delta_rank']


def _init(DATADIR):
    #### ==== CONST (blosum, multiprocessing, keys, etc) ==== ####
    VAL = math.floor(4 + (multiprocessing.cpu_count() / 1.5))
    N_CORES = VAL if VAL <= multiprocessing.cpu_count() else int(multiprocessing.cpu_count() - 2)

    MATRIXDIR = f'{DATADIR}Matrices/'
    ICSDIR = f'{DATADIR}ic_dicts/'
    AA_KEYS = [x for x in 'ARNDCQEGHILKMFPSTWYV']

    CHAR_TO_INT = dict((c, i) for i, c in enumerate(AA_KEYS))
    INT_TO_CHAR = dict((i, c) for i, c in enumerate(AA_KEYS))
    CHAR_TO_INT['X'] = -1
    CHAR_TO_INT['-'] = -1
    INT_TO_CHAR[-1] = '-'

    BG = np.loadtxt(f'{MATRIXDIR}bg.freq.fmt', dtype=float)
    BG = dict((k, v) for k, v in zip(AA_KEYS, BG))

    # BLOSUMS 50
    BL50 = {}
    _blosum50 = np.loadtxt(f'{MATRIXDIR}BLOSUM50', dtype=float).T
    for i, letter_1 in enumerate(AA_KEYS):
        BL50[letter_1] = {}
        for j, letter_2 in enumerate(AA_KEYS):
            BL50[letter_1][letter_2] = _blosum50[i, j]
    BL50_VALUES = {k: np.array([v for v in BL50[k].values()]) for k in BL50}
    # BLOSUMS 62
    BL62_DF = pd.read_csv(f'{MATRIXDIR}BLOSUM62', sep='\s+', comment='#', index_col=0)
    BL62 = BL62_DF.to_dict()
    BL62_VALUES = BL62_DF.drop(columns=['B', 'Z', 'X', '*'], index=['B', 'Z', 'X', '*'])
    BL62_VALUES = dict((x, BL62_VALUES.loc[x].values) for x in BL62_VALUES.index)

    # BLOSUMS 62 FREQS
    _blosum62 = np.loadtxt(f'{MATRIXDIR}BLOSUM62.freq_rownorm', dtype=float).T
    BL62FREQ = {}
    BL62FREQ_VALUES = {}
    for i, letter_1 in enumerate(AA_KEYS):
        BL62FREQ[letter_1] = {}
        BL62FREQ_VALUES[letter_1] = _blosum62[i]
        for j, letter_2 in enumerate(AA_KEYS):
            BL62FREQ[letter_1][letter_2] = _blosum62[i, j]
    #ICS_KL = pkl_load(ICSDIR + 'ics_kl_new.pkl')
    #ICS_SHANNON = pkl_load(ICSDIR + 'ics_shannon.pkl')
    #HLAS = ICS_SHANNON[9].keys()

    #return VAL, N_CORES, DATADIR, AA_KEYS, CHAR_TO_INT, INT_TO_CHAR, BG, BL62FREQ, BL62FREQ_VALUES, BL50, BL50_VALUES, BL62, BL62_VALUES, HLAS, ICS_KL, ICS_SHANNON
    return VAL, N_CORES, DATADIR, AA_KEYS, CHAR_TO_INT, INT_TO_CHAR, BG, BL62FREQ, BL62FREQ_VALUES, BL50, BL50_VALUES, BL62, BL62_VALUES


#VAL, N_CORES, DATADIR, AA_KEYS, CHAR_TO_INT, INT_TO_CHAR, BG, BL62FREQ, BL62FREQ_VALUES, BL50, BL50_VALUES, BL62, BL62_VALUES, HLAS, ICS_KL, ICS_SHANNON = _init(DATADIR)
VAL, N_CORES, DATADIR, AA_KEYS, CHAR_TO_INT, INT_TO_CHAR, BG, BL62FREQ, BL62FREQ_VALUES, BL50, BL50_VALUES, BL62, BL62_VALUES = _init(DATADIR)

In [6]:
encoding_matrix_dict = {'onehot': None,
                        'BL62LO': BL62_VALUES,
                        'BL62FREQ': BL62FREQ_VALUES,
                        'BL50LO': BL50_VALUES}

In [7]:
encoding = 'BL50LO'
pad_scale = None
df['len'] = df['Sequence'].apply(len)
window_size = 9
burnin_alphabet = 'ILVMFYW'
matrix_dim = 20
feature_cols = ['pseudoseq']

In [8]:
max_len = df['len'].max()
print(max_len)

35


In [9]:
def _get_burnin_mask(seq, max_len, motif_len, alphabet='ILVMFYW'):
    mask = torch.tensor([x in alphabet for i, x in enumerate(seq) if i < len(seq) - motif_len + 1]).float()
    return F.pad(mask, (0, (max_len - motif_len + 1) - len(mask)), 'constant', 0)

In [10]:
def _get_burnin_mask_batch(sequences, max_len, motif_len, alphabet='ILVMFYW'):
    return torch.stack([_get_burnin_mask(x, max_len, motif_len, alphabet) for x in sequences])

In [11]:
# X DIMENSIONS:
#     - Total number of sequences of the dataset (106949 sequences)
#     - Max-length padding: all of the sequences have been truncated to have the max-length (35 positions)
#     - One-hot encoding of the aa: 20 possible values of the BLOSUM encoding for each position in the sequence (20 aa)


x = encode_batch(df['Sequence'], max_len, encoding, pad_scale)
y = torch.from_numpy(df['BA'].values).float().view(-1, 1)

In [12]:
print(x.shape)

torch.Size([106949, 35, 20])


In [13]:
x = encode_batch(df['Sequence'], max_len, encoding, pad_scale)
y = torch.from_numpy(df['BA'].values).float().view(-1, 1)

print('X_dara original dimensions:')
print(x.shape, '\n')

# Creating the mask to allow selection of kmers without padding
x_mask = torch.from_numpy(df['len'].values) - window_size
range_tensor = torch.arange(max_len - window_size + 1).unsqueeze(0).repeat(len(x), 1)
# Mask for Kmers + padding
x_mask = (range_tensor <= x_mask.unsqueeze(1)).float().unsqueeze(-1)
# Creating another mask for the burn-in period+bool flag switch
burn_in_mask = _get_burnin_mask_batch(df['Sequence'].values, max_len, window_size, burnin_alphabet).unsqueeze(-1)
burn_in_flag = False
# Expand and unfold the sub kmers and the target to match the shape ; contiguous to allow for view operations
x_tensor = x.unfold(1, window_size, 1).transpose(2, 3) \
    .reshape(len(x), max_len - window_size + 1, window_size, matrix_dim).flatten(2, 3).contiguous()
y = y.contiguous()

print('X_tensor dimensions:')
print(x_tensor.shape, '\n')

# Add extra features
if len(feature_cols) > 0:
    # TODO: When you add more features you need to concatenate to x_pseudosequence and save it to self.x_features
    # these are NUMERICAL FEATURES like %Rank, expression, etc. of shape (N, len(feature_cols))
    # x_features = torch.from_numpy(df[feature_cols].values).float()

    extra_features_flag = True
else:
    x_features = torch.empty((len(x),))
    extra_features_flag = False

# TODO: Carlos, here you need to create the MHC feature vector and flatten it.
#       Basically, if you have the pseudo sequence in a column called 'pseudoseq' in your dataframe,
#       You can use my function encode_batch like
x_pseudoseq = encode_batch(df['pseudoseq'], 34, encoding, pad_scale)
print('X_pseudoseq dimensions after the first encoding:')
print(x_pseudoseq.shape, '\n')
    

# UNCOMMENT HERE WHEN YOU ARE DONE WITH THAT, check in a notebook that
# these dimension (N, 34*20) = (N, 680) are correct (you need to FLATTEN the vector using tensor.flatten(start_dim=1)
# then these should be working because my model forward() takes care of everything
    
x_pseudoseq = x_pseudoseq.flatten(start_dim=1)
print('X_pseudoseq dimensions after FLATTEN the vector:')
print(x_pseudoseq.shape, '\n')
x_features = x_pseudoseq
extra_features_flag = True

X_dara original dimensions:
torch.Size([106949, 35, 20]) 

X_tensor dimensions:
torch.Size([106949, 27, 180]) 

X_pseudoseq dimensions after the first encoding:
torch.Size([106949, 34, 20]) 

X_pseudoseq dimensions after FLATTEN the vector:
torch.Size([106949, 680]) 



In [14]:
print("Dimensions of the first sequence in X_tensor:")
print("     - 27 as the number of possible sequnces due to the sliding window (35-9+1)")
print("     - 180 as the number of 20 encoded aminoacids multiplied by the 9 length of the binding motif\n")
print(x_tensor[0].shape)
print(x_tensor[0])

Dimensions of the first sequence in X_tensor:
     - 27 as the number of possible sequnces due to the sliding window (35-9+1)
     - 180 as the number of 20 encoded aminoacids multiplied by the 9 length of the binding motif

torch.Size([27, 180])
tensor([[ -2.,  -2.,   2.,  ...,  15.,   2.,  -3.],
        [ -2.,  -3.,  -4.,  ...,  -3.,   2.,  -4.],
        [ -2.,  -2.,   2.,  ...,  -2.,  -1.,   1.],
        ...,
        [-12., -12., -12.,  ..., -12., -12., -12.],
        [-12., -12., -12.,  ..., -12., -12., -12.],
        [-12., -12., -12.,  ..., -12., -12., -12.]])


In [15]:
print("Total number of 9mer sequences in the training set:")
print(x_tensor.flatten(start_dim=0, end_dim=1).shape)

Total number of 9mer sequences in the training set:
torch.Size([2887623, 180])


In [16]:
print("Shape of the x_mask tensor (repeated 1 for each sequence 27 times according to the sliding window):")
print(x_mask.shape)

Shape of the x_mask tensor (repeated 1 for each sequence 27 times according to the sliding window):
torch.Size([106949, 27, 1])


## Individual PFR with masks (list)

In [31]:
# Define a function to calculate the mean position values of the flanking regions
def calculate_mean_flanking_positions(seq, mask, window_size=9):
    
    # Define output
    all_prev_pfr = []
    all_after_pfr = []
    
    # Previous PFR mask definition
    PFR_mask_before = 3 * torch.ones((27, 1))
    PFR_mask_before[:3,0] = torch.tensor([0, 1, 2], dtype=torch.float32)   # First three elements to 1 and 2
    
    # After PFR mask definition (according to their x_mask)
    PFR_mask_after = torch.clone(mask)*3

    # Modification of the previous values before 0
    PFR_mask_after[torch.where(PFR_mask_after == 0)[0][0]-3] = 2
    PFR_mask_after[torch.where(PFR_mask_after == 0)[0][0]-2] = 1
    PFR_mask_after[torch.where(PFR_mask_after == 0)[0][0]-1] = 0
    
    for i in range(0, seq.shape[0] - window_size + 1, 1):
        
        # Define the WS-mer
        peptide = seq[i:i + window_size]
        
        # Store the resulting PFR tensors
        all_prev_pfr.append(torch.sum(seq[i-int(PFR_mask_before[i]):i], dim=0, keepdim=True)/3)
        all_after_pfr.append(torch.sum(seq[i+window_size:i+window_size+int(PFR_mask_after[i])], dim=0, keepdim=True)/3)
    
    print(f"PFR mask for the 'previous' amino acids:\n {PFR_mask_before}")
    print(f"PFR mask for the 'after' amino acids:\n {PFR_mask_after}")
    
    return all_prev_pfr, all_after_pfr

In [32]:
seq1 = x[0,:]
x_mask1 = x_mask[0,:]
prev, after = calculate_mean_flanking_positions(seq1, x_mask1, window_size=9)

PFR mask for the 'previous' amino acids:
 tensor([[0.],
        [1.],
        [2.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]])
PFR mask for the 'after' amino acids:
 tensor([[3.],
        [3.],
        [2.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


In [19]:
print('Previous PFR for the first sequence:\n')

for element in prev:
    print(element)
    print("")

Previous PFR for the first sequence:

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

tensor([[-0.6667, -0.6667,  0.6667,  2.6667, -1.3333,  0.0000,  0.6667, -0.3333,
         -0.3333, -1.3333, -1.3333, -0.3333, -1.3333, -1.6667, -0.3333,  0.0000,
         -0.3333, -1.6667, -1.0000, -1.3333]])

tensor([[-1.3333, -1.6667, -0.6667,  1.3333, -2.0000, -0.6667, -0.3333, -1.6667,
         -1.3333, -0.6667,  0.3333, -1.3333, -0.3333, -1.3333, -1.6667, -1.0000,
         -0.6667, -2.3333, -1.3333, -1.0000]])

tensor([[-2.0000, -2.3333,  0.0000,  4.0000, -3.3333, -0.6667,  0.3333, -2.0000,
         -1.6667, -2.0000, -1.0000, -1.6667, -1.6667, -3.0000, -2.0000, -1.0000,
         -1.0000, -4.0000, -2.3333, -2.3333]])

tensor([[-1.6667, -0.6667, -0.6667,  1.0000, -3.0000,  0.0000,  0.0000, -2.3333,
         -1.3333, -1.6667, -0.6667,  0.6667, -1.0000, -2.6667, -2.0000, -1.0000,
         -1.0000, -3.3333, -2.0000, -2.0000]])

tensor([[-1.3333,  1.3333,  0.

In [20]:
print('After PFR for the first sequence:\n')

for element in after:
    print(element)
    print("")

After PFR for the first sequence:

tensor([[-1.6667, -1.0000, -1.0000, -1.0000, -2.6667,  0.3333,  1.0000, -3.0000,
          2.3333, -2.0000, -0.3333, -0.6667,  0.0000, -1.0000, -2.3333, -1.6667,
         -1.3333, -2.6667, -0.3333, -2.0000]])

tensor([[-1.3333, -1.0000, -1.3333,  0.0000, -2.6667,  0.6667,  3.0000, -3.3333,
         -1.0000, -2.0000, -0.3333, -0.3333, -0.3333, -1.6667, -2.0000, -1.6667,
         -1.0000, -2.6667, -1.6667, -1.6667]])

tensor([[-0.6667,  0.0000,  0.0000,  1.3333, -2.0000,  1.3333,  4.0000, -2.0000,
          0.0000, -2.6667, -2.0000,  0.6667, -1.3333, -2.0000, -0.6667, -0.6667,
         -0.6667, -2.0000, -1.3333, -2.0000]])

tensor([[-0.3333,  0.0000,  0.0000,  0.6667, -1.0000,  0.6667,  2.0000, -1.0000,
          0.0000, -1.3333, -1.0000,  0.3333, -0.6667, -1.0000, -0.3333, -0.3333,
         -0.3333, -1.0000, -0.6667, -1.0000]])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

tensor([[0., 0., 0., 0., 0., 0., 

## Individual PFR with masks (tensor)

In [21]:
# Define a function to calculate the mean position values of the flanking regions
def calculate_mean_flanking_positions_tensor(seq, mask, window_size=9):
    
    # Define output
    all_pfr = torch.empty((seq.shape[0] - window_size + 1, 2, seq.shape[1]))  # Initialize the output tensor
    
    # Previous PFR mask definition
    PFR_mask_before = 3 * torch.ones((27, 1))
    PFR_mask_before[:3, 0] = torch.tensor([0, 1, 2], dtype=torch.float32)   # First three elements to 1 and 2
    
    # After PFR mask definition (according to their x_mask)
    PFR_mask_after = torch.clone(mask) * 3

    # Modification of the previous values before 0 (only if zero exists)
    zero_indices = torch.where(PFR_mask_after == 0)[0]
    if zero_indices.numel() > 0:  # Check if there are any zero indices
        zero_index = zero_indices[0]  # Get the first zero index
        PFR_mask_after[zero_index - 3] = 2 if zero_index >= 3 else PFR_mask_after[zero_index - 3]
        PFR_mask_after[zero_index - 2] = 1 if zero_index >= 2 else PFR_mask_after[zero_index - 2]
        PFR_mask_after[zero_index - 1] = 0 if zero_index >= 1 else PFR_mask_after[zero_index - 1]

    
    for i in range(0, seq.shape[0] - window_size + 1, 1):
        
        # Define the WS-mer
        peptide = seq[i:i + window_size]
        
        # Store the resulting PFR tensors
        prev_pfr = torch.sum(seq[i - int(PFR_mask_before[i]):i], dim=0, keepdim=True) / 3
        after_pfr = torch.sum(seq[i + window_size:i + window_size + int(PFR_mask_after[i])], dim=0, keepdim=True) / 3

        # Store the PFR tensors in the output tensor
        all_pfr[i, 0] = prev_pfr
        all_pfr[i, 1] = after_pfr
    
    return all_pfr.flatten(start_dim=1)

In [22]:
seq1 = x[0,:]
x_mask1 = x_mask[0,:]
result = calculate_mean_flanking_positions_tensor(seq1, x_mask1, window_size=9)

print(result.shape)
print(result)

torch.Size([27, 40])
tensor([[  0.0000,   0.0000,   0.0000,  ...,  -2.6667,  -0.3333,  -2.0000],
        [ -0.6667,  -0.6667,   0.6667,  ...,  -2.6667,  -1.6667,  -1.6667],
        [ -1.3333,  -1.6667,  -0.6667,  ...,  -2.0000,  -1.3333,  -2.0000],
        ...,
        [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000],
        [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000],
        [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000]])


## All data PFR with masks (tensor)

In [23]:
# Define a function to calculate the mean position values of the flanking regions
def PFR_calculation(data, all_xmask, max_len, window_size=9):
    
    # Start the timer
    start_time = time.time()
    
    # Define output
    all_pfr = torch.empty((data.shape[0], max_len - window_size + 1, 2, 20))  # Initialize the output tensor
    
    for j in range(0, data.shape[0], 1):
        
        seq = data[j,:]
    
        # Previous PFR mask definition
        PFR_mask_before = 3 * torch.ones((max_len - window_size + 1, 1))
        PFR_mask_before[:3,0] = torch.tensor([0, 1, 2], dtype=torch.float32)   # First three elements to 1 and 2

        # After PFR mask definition (according to their x_mask)
        PFR_mask_after = torch.clone(all_xmask[j])*3

        # Modification of the previous values before 0 (only if zero exists)
        zero_indices = torch.where(PFR_mask_after == 0)[0]
        if zero_indices.numel() > 0:  # Check if there are any zero indices
            zero_index = zero_indices[0]  # Get the first zero index
            PFR_mask_after[zero_index - 3] = 2 if zero_index >= 3 else PFR_mask_after[zero_index - 3]
            PFR_mask_after[zero_index - 2] = 1 if zero_index >= 2 else PFR_mask_after[zero_index - 2]
            PFR_mask_after[zero_index - 1] = 0 if zero_index >= 1 else PFR_mask_after[zero_index - 1]
    
        for i in range(0, seq.shape[0] - window_size + 1, 1):

            # Define the WS-mer
            peptide = seq[i:i + window_size]

            # Store the resulting PFR tensors
            prev_pfr = torch.sum(seq[i - int(PFR_mask_before[i]):i], dim=0, keepdim=True) / 3
            after_pfr = torch.sum(seq[i + window_size:i + window_size + int(PFR_mask_after[i])], dim=0, keepdim=True) / 3

            # Store the PFR tensors in the output tensor
            all_pfr[j, i, 0] = prev_pfr
            all_pfr[j, i, 1] = after_pfr
            
    # End the timer
    end_time = time.time()
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(f"Function took {elapsed_time:.4f} seconds to run.")
    
    return all_pfr.flatten(start_dim=2)

In [24]:
result = PFR_calculation(x, x_mask, max_len, window_size=9)

Function took 290.0358 seconds to run.


Around 4,83 minutes to run

In [25]:
result.shape, result[0]

(torch.Size([106949, 27, 40]),
 tensor([[  0.0000,   0.0000,   0.0000,  ...,  -2.6667,  -0.3333,  -2.0000],
         [ -0.6667,  -0.6667,   0.6667,  ...,  -2.6667,  -1.6667,  -1.6667],
         [ -1.3333,  -1.6667,  -0.6667,  ...,  -2.0000,  -1.3333,  -2.0000],
         ...,
         [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000],
         [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000],
         [-12.0000, -12.0000, -12.0000,  ...,   0.0000,   0.0000,   0.0000]]))